In [2]:
import os
import sys
import numpy as np
from scipy.special import logsumexp
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

### What do we look for?

We look for the optimal design $d^*$ such as:
\begin{equation}
 d^* = \arg \max_{d}{U(d)}
\end{equation}

### Define the utility function

We define a utility function $U$ as the mutual information of $\theta$ and $Y\mid d$:
\begin{equation}
    U(d) = I(\Theta; Y\mid d)
\end{equation}
Y being the possible observations (e.g., behavioral outputs).

### Bayesian revision of belief regarding the parameters
Supposing that an experiment was carried out with design $d^*$, and an outcome $y_t$ was observed, the prior over $\theta$ at $t$, $p_t(\theta)$, is revised the following way:
\begin{equation}
     p_{t+1}(\theta) = \dfrac{
      p(y_t \mid \theta, d^*) p_t(\theta)
     }{
        \int p(y_t \mid \theta, d^*) p_t(\theta) d\theta
     }
\end{equation}

### Exchanging definitions

\begin{equation}
U(d) = \int \int u(d, \theta, y) p(y \mid \theta, d) p(\theta) d\theta dy
\end{equation}
if $U(d) = I(\Theta; Y\mid d)$ then $u(d, \theta, y) = \log \left( \frac{p (\theta \mid y, d)}{p(\theta)} \right)$
which means

\begin{equation}
U(d) = I(\Theta; Y\mid d) = \log \left( \frac{p (\theta \mid y, d)}{p(\theta)} \right) p(y \mid \theta, d) p(\theta) d\theta dy
\end{equation}

### With steps...
\begin{equation}
\begin{split}
U(d) &= I(\Theta; Y\mid d) \\
&= H(\Theta) - H(\Theta \mid Y, d)\\
\end{split}
\end{equation}

\begin{equation}
\begin{split}
H(\Theta) &= - \int p(\theta) \log p(\theta) d\theta \\
&= - \int p(\theta) \log p(\theta) \int p(y \mid \theta, d) dy d\theta \\
&= - \int \int p(\theta) \log p(\theta) p(y \mid \theta, d) dy d\theta \\
&= - \int \int p(\theta) p(y \mid \theta, d) \log p(\theta) dy d\theta \\
\end{split}
\end{equation}

\begin{equation}
\begin{split}
H(\Theta \mid Y, d) &= - \int p(y \mid d) \int p( \theta \mid y, d) \log p(\theta \mid y, d) dy d\theta \\
&=  - \int p(y \mid d) \int p(\theta) p( y \mid \theta, d) \frac{1}{p(y \mid d)}   \log p(\theta \mid y, d) dy d\theta \\
&=  - \int \int p(y \mid d) p(\theta) p( y \mid \theta, d) \frac{1}{p(y \mid d)}   \log p(\theta \mid y, d) dy d\theta \\
&= - \int \int p(\theta) p( y \mid \theta, d)   \log p(\theta \mid y, d) dy d\theta \\
\end{split}
\end{equation}

\begin{equation}
\begin{split}
U(d) &= \int \int - p(\theta) \log p(\theta) p(y \mid \theta, d) + p(\theta) p( y \mid \theta, d)   \log p(\theta \mid y, d) dy d\theta \\
&= \int \int - p(\theta) p(y \mid \theta, d) \log p(\theta) + p(\theta) p( y \mid \theta, d)   \log p(\theta \mid y, d) dy d\theta \\
&= \int \int p(\theta) p(y \mid \theta, d) [ \log p (\theta \mid y, d) - \log p(\theta) ] dy d\theta \\
&= \int \int p(\theta) p(y \mid \theta, d) \log \frac{p (\theta \mid y, d)}{p(\theta)} dy d\theta \\
\end{split}
\end{equation}

### Implementation

Implementation follows what has been done in: https://github.com/adopy/adopy

Actually, in the implementation, we can use the fact that the mutual information is symetric. Indeed, for any pair of random variables $X$ and $Y$, we have:

\begin{equation}
I(X; Y) = H(X) - H(X \mid Y) = I(Y; X) = H(Y) - H(Y \mid X)
\end{equation}

which means that in our case, we have:
\begin{equation}
I(\Theta; Y \mid d) = I(Y \mid d; \Theta) = H(Y \mid d) - H(Y \mid \Theta, d) 
\end{equation}

The marginal entropy is:

\begin{equation}
   H(Y \mid d) = - \int p(y \mid d) \log p(y \mid d) dy
\end{equation}

In our implementation, we use log probabilities instead of proabilities, and $Y$ is a discreet space:
\begin{equation}
   H(Y \mid d) =  - \sum_{y \in Y} \exp[\log p(y \mid d)] \log p(y \mid d)
\end{equation}

The conditional entropy of $Y$ given the outcome random variable $\Theta$ and design $d$ is:
\begin{equation}
        H(Y \mid \Theta, d) = - \int p(\theta) \int p(y\mid \theta, d) \log p(y\mid \theta, d) dyd\theta
\end{equation}

In our implementation, we use log probabilities instead of proabilities, and $Y$ is a discreet space, and since we use a grid exploration technique, $\Theta$ is also considered to be a discreet space:
\begin{equation}
H(Y \mid \Theta, d) = - \sum_{\theta \in \Theta} \exp[\log p(\theta)] \sum_{y\in Y} \exp[ \log p(y\mid \theta, d)] \log p(y\mid \theta, d)
\end{equation}

### From parameter estimation to model selection

\begin{equation}
     U(d) = \sum_m p(m) \int \int u(d, \theta_m, y_m) p(y_m \mid \theta_m, d) p_{\theta_m} dy_m d\theta_m,
\end{equation}
where m = {1, 2, ..., K} is one of a set of K models 

\begin{equation}
\begin{split}
U(d) &= I(M; Y\mid d) \\
u(d, \theta_m, y_m) &= \log \frac{p(m \mid y, d)}{p(m)}
\end{split}
\end{equation}

where $I(M; Y |d)$ is the mutual information between the model random variable M and the outcome random variable conditional upon design d, Y |d. 

$p(m \mid y, d)$ is the posterior model probability of model $m$ obtained by Bayes rule as:
\begin{equation}
p(m|y, d) = \frac{p(y|m, d)p(m)}{p(y|d)} 
\end{equation}

where
\begin{equation}
p(y|m, d) = \int p(y| \theta_m, d)p(\theta_m) d\theta_m 
\end{equation}

\begin{equation}
p(y|d) = \sum_m p(y|m, d)p(m)
\end{equation}

### Bayesian update of belief regarding the model

\begin{equation}
p_{t+1}(m) = \frac{p_1(m)}{\sum_{k=1}^K p_1(k) BF_{(k, m)}(y_t \mid d^*)}
\end{equation}

\begin{equation}
BF(k,m) (y \mid d) = \frac{\int p(y \mid \theta_k, d) p(\theta_k) d\theta_k}{\int p(y \mid \theta_m, d) p(\theta_m) d\theta_m} 
\end{equation}

### Implementation
\begin{equation}
\log p_{t+1}(m) = \log p_1(m) - \log \sum_{k=1}^K \exp \left(log p_1(k) + \log \sum_{\theta \in \Theta_k} \exp[\log p(y\mid \theta, d) + \log p (\theta)]  - \log \sum_{\theta \in \Theta_m} \exp[\log p(y\mid \theta, d) + \log p (\theta)]  \right)
\end{equation}